Importing Libraries


In [71]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
import random
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chirag\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chirag\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading Dataset


In [86]:
with open("D://Study Material//My AI's//Mark 2//intents.json", 'r', encoding='utf8') as file:
    data = json.load(file)

Initializing Dependent, Independent, Lemmatizer, Stemmer Variables


In [87]:
intents = data['intents']
X = []
y = []
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

This Function is for the preprocessing of data before fitting it in model. Various NLP techniques are used here to preprocess the data


In [88]:
def preprocess_text(text):
    # Normalize text (convert to lowercase and remove non-alphanumeric characters)
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)   #Substitute the matched string in text with space.

    # Stemming
    stemmed_words = [stemmer.stem(word) for word in text.split()]

    #tokenize
    tokenized_words = word_tokenize(text)

    # Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokenized_words]   #changing, Changed---->change

    # Join the preprocessed words back into a single string
    processed_text = " ".join(lemmatized_words)
    return processed_text

Separating the Dependent and Independent values after preprocessing the data and storing it in X(Independent) and y(Dependent) variable


In [89]:
for intent in intents:
    tag = intent['tag']
    patterns = intent['patterns']
    for pattern in patterns:
        processed_pattern = preprocess_text(pattern)
        X.append(processed_pattern)
        y.append(tag)

Initializing the Vectorizer


In [90]:
vectorizer = TfidfVectorizer()

Splitting the data into training and testing data, here the ration is 80:20


In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Create the model and fit the training data in the model


In [104]:
model = Pipeline([
    ('tfidf', vectorizer),
    ('svm', SVC(kernel='linear', C=1.5, probability=True))
])

# Fit the model on the training data
model.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('svm', SVC(C=1.5, kernel='linear', probability=True))])

Finding the accuracy of model


In [105]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Intent Recognition Accuracy: {accuracy*100}%")
#Initially the model accuracy is 57.14285714285714%, but after adding more data to the intents.json file, the accuracy increased to 85.71428571428571%

Intent Recognition Accuracy: 73.33333333333333%


In [135]:
# List to store conversation history
conversation_history = []

Function for Recognising the Intent of user


In [130]:
def recognize_intent(message):
    processed_message = preprocess_text(message)
    intent = model.predict([processed_message])[0]
    print("Intent = ",intent)
    print("Probability = ",model.predict_proba([message]))
    return intent

Funtion for getting response on the basis for intent


In [131]:
def get_bot_response(intent, history):
    for intent_data in intents:
        if intent_data['tag'] == intent:
            responses = intent_data['responses']
            # Modify the response generation based on the conversation history
            # Example logic: Access the last user input and provide a relevant response
            last_user_input = history[-1]['patterns']
            if last_user_input == 'help':
                return "I see you need help. How can I assist you?"

            return random.choice(responses)

Funtion to initialize the chatbot with some personalized features


In [132]:
def chatbot():
    
    print("Bot: Hello! {User} How can I assist you?")
    
    while True:
        user_input = input("{User}: ")
        if user_input.lower() in ["stop", "quit"]:
            print("Bot: Goodbye! Have a great day!")
            break
        elif user_input == "":
            print("Bot: You have not entered anything, Please enter a message.")
            continue

        recognized_intent = recognize_intent(user_input)
        conversation_history.append({'tag': recognized_intent, 'patterns': user_input})

        last_user_input = conversation_history[-1]['patterns']
        last_intent = conversation_history[-1]['tag']

        bot_response = get_bot_response(last_intent, conversation_history)

        conversation_history[-1].update(responses= bot_response)

        print("Bot:", bot_response)

Here we call the chatbot function and at the end the conversation history saved and in near future we'll gonna use it for traing our model


In [133]:
def Start():
    # heyhey = input(f"Do you want me to call you {User} from now on or I call you BOSS? ----> ")
    # if heyhey.lower() == "boss":
    #   User = "BOSS"
    # elif heyhey.lower() in ["yes", f'{User}']:
    #   User = User
    # else:
    #    print("Please enter a valid input")
    #    Start(User)
      
    print(f"I'm your personal assistant. I can help you with your queries. If you want to stop this conversation, just type 'stop' or 'quit' and I'll stop our conversation.")
    chatbot()
    print("\n*****This is your conversation history, which we'll gonna use to train our model*****\n\n","Intent : ",
      [i['tag'] for i in conversation_history],"\n","User Input : ",
      [i['patterns'] for i in conversation_history],"\n","Bot Responses : ",
      [i['responses'] for i in conversation_history]
    )

START.........Run the following cell

In [136]:
Start()

I'm your personal assistant. I can help you with your queries. If you want to stop this conversation, just type 'stop' or 'quit' and I'll stop our conversation.
Bot: Hello! {User} How can I assist you?
Intent =  greeting
Probability =  [[0.06304458 0.01878875 0.06794946 0.5440037  0.02410129 0.02894321
  0.04461576 0.03078252 0.03199748 0.0730461  0.05332812 0.01939904]]
Bot: Hi, hi, hi! What's the latest gossip?
Bot: Goodbye! Have a great day!

*****This is your conversation history, which we'll gonna use to train our model*****

 Intent :  ['greeting'] 
 User Input :  ['hello'] 
 Bot Responses :  ["Hi, hi, hi! What's the latest gossip?"]


In [ ]:
#Trial for merging branch